# Visualizing a Gensim model

To illustrate how to use [`pyLDAvis`](https://github.com/bmabey/pyLDAvis)'s gensim [helper funtions](https://pyldavis.readthedocs.org/en/latest/modules/API.html#module-pyLDAvis.gensim) we will create a model from the [20 Newsgroup corpus](http://qwone.com/~jason/20Newsgroups/). Minimal preprocessing is done and so the model is not the best. However, the goal of this notebook is to demonstrate the helper functions.

## Downloading the data

In [1]:
%%bash
mkdir -p data
pushd data
if [ -d "20news-bydate-train" ]
then
  echo "The data has already been downloaded..."
else
  wget http://qwone.com/%7Ejason/20Newsgroups/20news-bydate.tar.gz
  tar xfv 20news-bydate.tar.gz
  rm 20news-bydate.tar.gz
fi
echo "Lets take a look at the groups..."
ls 20news-bydate-train/
popd

~/Documents/Reddit_NLP/data ~/Documents/Reddit_NLP
20news-bydate-test/
20news-bydate-test/alt.atheism/
20news-bydate-test/alt.atheism/53265
20news-bydate-test/alt.atheism/53339
20news-bydate-test/alt.atheism/53260
20news-bydate-test/alt.atheism/53340
20news-bydate-test/alt.atheism/53333
20news-bydate-test/alt.atheism/53302
20news-bydate-test/alt.atheism/53313
20news-bydate-test/alt.atheism/53293
20news-bydate-test/alt.atheism/53297
20news-bydate-test/alt.atheism/53315
20news-bydate-test/alt.atheism/53320
20news-bydate-test/alt.atheism/53324
20news-bydate-test/alt.atheism/53328
20news-bydate-test/alt.atheism/53325
20news-bydate-test/alt.atheism/53322
20news-bydate-test/alt.atheism/53326
20news-bydate-test/alt.atheism/53261
20news-bydate-test/alt.atheism/53327
20news-bydate-test/alt.atheism/53329
20news-bydate-test/alt.atheism/53321
20news-bydate-test/alt.atheism/53068
20news-bydate-test/alt.atheism/53338
20news-bydate-test/alt.atheism/53257
20news-bydate-test/alt.atheism/53262
20news-by

--2020-08-16 21:29:50--  http://qwone.com/%7Ejason/20Newsgroups/20news-bydate.tar.gz
Resolving qwone.com (qwone.com)... 108.20.201.166
Connecting to qwone.com (qwone.com)|108.20.201.166|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14464277 (14M) [application/x-gzip]
Saving to: ‘20news-bydate.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  298K 47s
    50K .......... .......... .......... .......... ..........  0%  668K 34s
   100K .......... .......... .......... .......... ..........  1% 1.66M 25s
   150K .......... .......... .......... .......... ..........  1% 1.71M 21s
   200K .......... .......... .......... .......... ..........  1% 1.11M 19s
   250K .......... .......... .......... .......... ..........  2% 1.67M 17s
   300K .......... .......... .......... .......... ..........  2% 1.76M 16s
   350K .......... .......... .......... .......... ..........  2% 1.90M 15s
   400K .......... .......... .......... .......... ..

## Exploring the dataset

Each group dir has a set of files:

In [2]:
ls -lah data/20news-bydate-train/sci.space | tail  -n 5

-rw-r--r--  1 nick nick 1.5K Mar 18  2003 61250
-rw-r--r--  1 nick nick  889 Mar 18  2003 61252
-rw-r--r--  1 nick nick 1.2K Mar 18  2003 61264
-rw-r--r--  1 nick nick 1.7K Mar 18  2003 61308
-rw-r--r--  1 nick nick 1.4K Mar 18  2003 61422


Lets take a peak at one email:

In [3]:
!head data/20news-bydate-train/sci.space/61422 -n 20

From: ralph.buttigieg@f635.n713.z3.fido.zeta.org.au (Ralph Buttigieg)
Subject: Why not give $1 billion to first year-lo
Organization: Fidonet. Gate admin is fido@socs.uts.edu.au
Lines: 34

Original to: keithley@apple.com
G'day keithley@apple.com

21 Apr 93 22:25, keithley@apple.com wrote to All:

 kc> keithley@apple.com (Craig Keithley), via Kralizec 3:713/602


 kc> But back to the contest goals, there was a recent article in AW&ST
about a
 kc> low cost (it's all relative...) manned return to the moon.  A General
 kc> Dynamics scheme involving a Titan IV & Shuttle to lift a Centaur upper
 kc> stage, LEV, and crew capsule.  The mission consists of delivering two
 kc> unmanned payloads to the lunar surface, followed by a manned mission.
 kc> Total cost:  US was $10-$13 billion.  Joint ESA(?)/NASA project was


## Loading and tokenizing the corpus

In [4]:
from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk
import pandas as pd

In [8]:
# quick and dirty....
EMAIL_REGEX = re.compile(r"[a-z0-9\.\+_-]+@[a-z0-9\._-]+\.[a-z]*")
FILTER_REGEX = re.compile(r"[^a-z '#]")
TOKEN_MAPPINGS = [(EMAIL_REGEX, "#email"), (FILTER_REGEX, ' ')]

def tokenize_line(line):
    res = line.lower()
    for regexp, replacement in TOKEN_MAPPINGS:
        res = regexp.sub(replacement, res)
    return res.split()
    
def tokenize(lines, token_size_filter=2):
    tokens = fp.mapcat(tokenize_line, lines)
    return [t for t in tokens if len(t) > token_size_filter]
    

def load_doc(filename):
    group, doc_id = filename.split('/')[-2:]
    with open(filename, errors='ignore') as f:
        doc = f.readlines()
    return {'group': group,
            'doc': doc,
            'tokens': tokenize(doc),
            'id': doc_id}


docs = pd.DataFrame(list(map(load_doc, glob('data/20news-bydate-train/*/*')))).set_index(['group','id'])
docs

doc  \
group            id                                                         
rec.sport.hockey 53802  [From: shah@pitt.edu (Ravindra S Shah)\n, Subj...   
                 53660  [Organization: Central Michigan University\n, ...   
                 53706  [From: jrmst8+@pitt.edu (Joseph R Mcdonald)\n,...   
                 53868  [From: hammerl@acsu.buffalo.edu (Valerie S. Ha...   
                 53830  [From: gp2011@andy.bgsu.edu (George Pavlic)\n,...   
...                                                                   ...   
comp.graphics    38570  [From: freemant@dcs.glasgow.ac.uk (Toby Freema...   
                 38581  [From: dstampe@psych.toronto.edu (Dave Stampe)...   
                 38632  [From: wdm@world.std.com (Wayne Michael)\n, Su...   
                 38595  [From: srlnjal@grace.cri.nz\n, Subject: CorelD...   
                 38217  [From: rob@rjck.UUCP (Robert J.C. Kyanko)\n, S...   

                                                                   tokens  
group            id                                                        
rec.sport.hockey 53802  [from, #email, ravindra, shah, subject, nords,...  
                 53660  [organization, central, michigan, university, ...  
                 53706  [from, #email, joseph, mcdonald, subject, plus...  
                 53868  [from, #email, valerie, hammerl, subject, goal...  
                 53830  [from, #email, george, pavlic, subject, matt, ...  
...                                                                   ...  
comp.graphics    38570  [from, #email, toby, freeman, tjf, oct, subjec...  
                 38581  [from, #email, dave, stampe, subject, fast, po...  
                 38632  [from, #email, wayne, michael, subject, adobe,...  
                 38595  [from, #email, subject, coreldraw, bitmap, sco...  
                 38217  [from, #email, robert, kyanko, subject, vga, g...  

[11314 rows x 2 columns]

## Creating the dictionary, and bag of words corpus

In [6]:

def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, additional_stopwords=set(), no_below=5, no_above=0.5):
  print('Building dictionary...')
  dictionary = Dictionary(docs)
  stopwords = nltk_stopwords().union(additional_stopwords)
  stopword_ids = map(dictionary.token2id.get, stopwords)
  dictionary.filter_tokens(stopword_ids)
  dictionary.compactify()
  dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
  dictionary.compactify()

  print('Building corpus...')
  corpus = [dictionary.doc2bow(doc) for doc in docs]

  return dictionary, corpus



In [7]:
dictionary, corpus = prep_corpus(docs['tokens'])

Building dictionary...
Building corpus...


In [9]:
MmCorpus.serialize('newsgroups.mm', corpus)
dictionary.save('newsgroups.dict')

## Fitting the LDA model

In [10]:
%%time
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=50, passes=10)
                                      
lda.save('newsgroups_50_lda.model')

CPU times: user 21min 17s, sys: 20min 28s, total: 41min 45s
Wall time: 3min 14s


## Visualizing the model with pyLDAvis

Okay, the moment we have all been waiting for is finally here!  You'll notice in the visualization that we have a few junk topics that would probably disappear after better preprocessing of the corpus. This is left as an exercises to the reader. :)

In [11]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [12]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

## Fitting the HDP model

We can both visualize LDA models as well as gensim HDP models with pyLDAvis.

The difference between HDP and LDA is that HDP is a non-parametric method. Which means that we don't need to specify the number of topics. HDP will fit as many topics as it can and find the optimal number of topics by itself.

In [ ]:
%%time
# The optional parameter T here indicates that HDP should find no more than 50 topics
# if there exists any.
hdp = models.hdpmodel.HdpModel(corpus, dictionary, T=50)
                                      
hdp.save('newsgroups_hdp.model')

## Visualizing the HDP model with pyLDAvis

As for the LDA model, in order to prepare the visualization you only need to pass it your model, the corpus, and the associated dictionary.

In [ ]:
vis_data = gensimvis.prepare(hdp, corpus, dictionary)
pyLDAvis.display(vis_data)